In [63]:
pip install nltk scikit-learn streamlit

Note: you may need to restart the kernel to use updated packages.


In [64]:
import random
import json
import os
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Handling SSL certificate issue (if any)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arun1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [65]:
# Define chatbot intents (Smart Farming + General Queries)
intents = [
    {"tag": "greeting", "patterns": ["Hi", "Hello", "Hey", "What's up", "How are you"],
     "responses": ["Hi there!", "Hello!", "Hey!", "Nothing much.", "I'm fine, thank you!"]},

    {"tag": "goodbye", "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
     "responses": ["Goodbye!", "See you later!", "Take care!"]},

    {"tag": "thanks", "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
     "responses": ["You're welcome!", "No problem!", "Glad I could help!"]},

    {"tag": "about", "patterns": ["What can you do?", "Who are you?", "What are you?", "What is your purpose?"],
     "responses": ["I am a chatbot for Smart Farming!", "I assist in answering farming-related queries.", "I can help with agriculture, crop health, and weather advice."]},

    {"tag": "help", "patterns": ["Help", "I need help", "Can you help me?", "What should I do?"],
     "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]},

    {"tag": "age", "patterns": ["How old are you?", "What's your age?"],
     "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]},

    {"tag": "weather", "patterns": ["What's the weather like?", "How's the weather today?"],
     "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]},

    {"tag": "budget", "patterns": ["How can I make a budget?", "What's a good budgeting strategy?", "How do I create a budget?"],
     "responses": ["To make a budget, track your income and expenses.", "Use the 50/30/20 rule: 50% essentials, 30% wants, 20% savings.", "Set financial goals and allocate money wisely."]},

    {"tag": "credit_score", "patterns": ["What is a credit score?", "How do I check my credit score?", "How can I improve my credit score?"],
     "responses": ["A credit score represents your creditworthiness.", "You can check your credit score on sites like Credit Karma.", "Pay bills on time and reduce debt to improve your score."]},

    {"tag": "soil_health", "patterns": ["How to improve soil health?", "Best ways to maintain soil?", "How can I increase soil fertility?"],
     "responses": ["Use organic compost and rotate crops.", "Avoid excessive chemical fertilizers.", "Plant cover crops to improve soil quality."]},

    {"tag": "pest_control", "patterns": ["How to control pests naturally?", "What are organic pest control methods?", "How can I reduce pests in farming?"],
     "responses": ["Use neem oil and companion planting.", "Crop rotation helps prevent pest infestation.", "Introduce natural predators like ladybugs."]},

    {"tag": "fertilizer", "patterns": ["What fertilizer should I use for wheat?", "Best fertilizer for rice?", "Which fertilizer improves crop yield?"],
     "responses": ["For wheat, use Nitrogen-based fertilizers like Urea.", "Rice grows well with Potassium-based fertilizers.", "Balanced fertilizers (NPK) are great for overall growth."]},

    {"tag": "crop_rotation", "patterns": ["Why is crop rotation important?", "Benefits of crop rotation?", "How does crop rotation help?"],
     "responses": ["It prevents soil depletion and improves fertility.", "Crop rotation reduces pests and diseases.", "It enhances biodiversity and sustainable farming."]}
]


In [66]:
# Save memory file for chatbot
MEMORY_FILE = "chatbot_memory.json"

# Load memory if available
if os.path.exists(MEMORY_FILE):
    with open(MEMORY_FILE, "r") as file:
        chatbot_memory = json.load(file)
else:
    chatbot_memory = {}


In [67]:
# Create the vectorizer and classifier
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

# Prepare the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)



In [68]:
# Train the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

# Chatbot function
def chatbot(input_text):
    input_text_lower = input_text.lower()
    
    # Check memory for learned responses
    if input_text_lower in chatbot_memory:
        return chatbot_memory[input_text_lower]
    
    # Predict category
    input_vector = vectorizer.transform([input_text])
    predicted_tag = clf.predict(input_vector)[0]
    
    # Search for response
    for intent in intents:
        if intent["tag"] == predicted_tag:
            return random.choice(intent["responses"])
    
    return None


In [69]:
# Running chatbot 
while True:
    user_query = input("\nYou: ")  # User input
    if user_query.lower() in ["exit", "quit", "bye"]:
        print("\nChatbot: Goodbye! Have a great day! 😊")
        break

    response = chatbot(user_query)

    # If chatbot doesn't recognize the input, ask for help
    if response is None:
        new_response = input(f"\nChatbot: I don't know the answer to '{user_query}'. Can you teach me? ")
        if new_response:
            chatbot_memory[user_query.lower()] = new_response  # Save new response
            with open(MEMORY_FILE, "w") as file:
                json.dump(chatbot_memory, file)  # Save to memory
            print("\nChatbot: Thank you! I have learned a new response.")
            response = new_response  # Use the new response

    print("\nChatbot:", response)  # Display chatbot response


You:  hi



Chatbot: Hi there!



You:  fertilizer



Chatbot: Rice grows well with Potassium-based fertilizers.



You:  ai


...


Chatbot: I can help with agriculture, crop health, and weather advice.



You:  what is this



Chatbot: I am a chatbot for Smart Farming!



You:  bye



Chatbot: Goodbye! Have a great day! 😊
